In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from find_peak_slope import *
from zacLib import awgn
%matplotlib Qt5

In [8]:
find_peak_slope_from_rd('siga.rd',250,25,1,0)

[  7463  12473  17576 182559 187291 196280 204599 205408 208712 213293
 214475 218970 220146 228297 230351 239542 241285 246413 275408 291404
 300061 300660 303250 306610 307703 308949 311338 315673 316304 321356
 322097 323163 324478 330354 331217 331985 332425 338528 341810 342974] [  7670  12825  17811 183506 188965 196928 204842 207189 208908 214150
 215183 219447 220939 229773 231110 240881 242053 249139 278165 297526
 300410 301053 304029 307449 308547 310702 312815 316053 316764 321828
 322866 323586 325612 330842 331729 332188 333301 340344 342707 343667]


,avg_frq,max_frq,slope band,half_band
0,7566,7580,207,133
1,12649,12632,352,177
2,17693,17684,235,168
3,183032,183223,947,923
4,188128,188657,1674,1610
5,196604,196521,648,602
6,204720,204696,243,165
7,206298,206116,1781,1756
8,208810,208818,196,184
9,213721,213461,857,729


In [3]:
def bell(n,h,sigma=1):
    return h*np.exp(-np.arange(-n//2,n//2)**2/2/sigma**2)
def square(n,h,alpha=0,ripple=0):
    if alpha>n//2: alpha = n//2
    s = np.zeros(n)
    s[0:alpha] = np.cos(np.linspace(np.pi,np.pi*2,alpha))+1
    s[n-alpha:n] = np.cos(np.linspace(0,np.pi,alpha))+1
    s[alpha:n-alpha] = 2+ripple*np.sin(np.linspace(0,12*np.pi,n-2*alpha))
    return 0.5*h*s

def stair(n,h1,h2,alpha=0,sigma=1,ripple=0):
    return bell(n,h1,sigma)+square(n,h2,alpha,ripple)


In [4]:
n=int(8000)
x = np.zeros(n)
x[100:1000] += square(900,1,alpha=0,ripple=0)
x[1100:2000] += square(900,1,alpha=100,ripple=0)
x[2100:3000] += square(900,1,alpha=300,ripple=0)
x[3100:4000] += square(900,1,alpha=450,ripple=0)
x[4100:5000] += square(900,0.5,alpha=0,ripple=0)
x[5100:6000] += square(900,0.5,alpha=100,ripple=0)
x[6100:7000] += square(900,0.5,alpha=300,ripple=0)
x[7100:8000] += square(900,0.5,alpha=450,ripple=0)

# stairs
# x[100:1000] = stair(900,1,0.1,alpha=100,sigma=30,ripple=0)
# x[1100:2000] = stair(900,0.4,1,alpha=200,sigma=10,ripple=0)
# x[2100:3000] = stair(900,1,1,alpha=30,sigma=50,ripple=0)
# x[3100:4000] = stair(900,1,1,alpha=300,sigma=30,ripple=0)
nx = awgn(x,None,0.05)
# plt.figure()
# plt.plot(nx,'.',markersize=1)
# plt.plot(x)

# plt.show()
